In [ ]:
# === colab configuration ===
# p.s. when training on colab, weights are saved on Drive (directory DSCAS/weights).
# p.p.s. skip this cell if running demo file locally!

! pip install torch-summary
from google.colab import drive
import sys

# setting paths
repository_path = '/content/deep-seasonal-color-analysis-system/'
dataset_path = repository_path + 'headsegmentation_dataset_ccncsa/'
dataset_path_drive = '/content/drive/MyDrive/DSCAS/headsegmentation_dataset_ccncsa/'
weights_path = repository_path + 'models/weights/'
weights_path_drive = '/content/drive/MyDrive/DSCAS/weights/'
sys.path.insert(0, repository_path)

# cloning project repository and downloading dataset
drive.mount('/content/drive')
! test ! -d $repository_path && git clone https://github.com/mrcmich/deep-seasonal-color-analysis-system.git
! test ! -d $dataset_path && cp -R $dataset_path_drive $dataset_path
%cd $repository_path

# setting branch and pulling updates
branch = 'main__fastscnn_hyperparameters_tuning'
! git checkout $branch
! git pull origin $branch

executing_on_colab = True

In [ ]:
import torch
from torch import nn
from torchvision import ops
import torchvision.transforms as T
from sklearn.model_selection import train_test_split
from models import dataset, training_and_testing
from models.local.FastSCNN.models import fast_scnn
from metrics_and_losses import metrics
from utils import segmentation_labels, utils
import matplotlib.pyplot as plt
from palette_classification import color_processing
import torchsummary
from models.config import *

try:
  executing_on_colab
except NameError:
  executing_on_colab = False

In [ ]:
# local configuration
if executing_on_colab is False:
  weights_path = 'models/weights/'
  dataset_path = ROOT_DIR + 'headsegmentation_dataset_ccncsa/'

In [ ]:
# defining transforms
tH, tW = 256, 256
mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225] # from ImageNet
image_transform = T.Compose([T.Resize((tH, tW)), T.Normalize(mean, std)])
target_transform = T.Compose([T.Resize((tH, tW))])

# fetching dataset
n_classes = len(segmentation_labels.labels)
img_paths, label_paths = dataset.get_paths(dataset_path, file_name='training.xml')
X_train, X_test, Y_train, Y_test = train_test_split(img_paths, label_paths, test_size=0.20, random_state=99, shuffle=True)
train_dataset = dataset.MyDataset(X_train, Y_train, image_transform, target_transform)
test_dataset = dataset.MyDataset(X_test, Y_test, image_transform, target_transform)

# setting up model and fixed (initially) hyperparameters
device = 'cuda' if torch.cuda.is_available() else 'cpu'
n_epochs = 10
batch_size = 32
score_fn = metrics.batch_mIoU
learning_rate = 0.01
class_weights = torch.tensor([0.3762, 0.9946, 0.9974, 0.9855, 0.7569, 0.9140, 0.9968, 0.9936, 0.9989, 0.9893, 0.9968])

In [ ]:
# === selecting best loss function ===

results = []

# in fase di valutazione
loss_fn_candidates = [
    nn.CrossEntropyLoss(),
    nn.CrossEntropyLoss(weight=class_weights)]

for loss_fn in loss_fn_candidates:
    model = fast_scnn.FastSCNN(n_classes)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    partial_results = training_and_testing.train_model(
        device, model, train_dataset, batch_size, n_epochs, score_fn, loss_fn, optimizer, 
        lr_scheduler=None, evaluate=True)
    results.append(partial_results)

In [ ]:
# === selecting best optimizer ===

results = []
loss_fn = nn.CrossEntropyLoss()

models = [
    fast_scnn.FastSCNN(n_classes),
    fast_scnn.FastSCNN(n_classes),
    fast_scnn.FastSCNN(n_classes),
]

# in fase di valutazione
optimizer_candidates = [
    torch.optim.SGD(models[0].parameters(), lr=learning_rate),
    torch.optim.Adam(models[1].parameters(), lr=learning_rate),
    torch.optim.AdamW(models[2].parameters(), lr=learning_rate)]

for i, optimizer in enumerate(optimizer_candidates):
    model = models[i]
    partial_results = training_and_testing.train_model(
        device, model, train_dataset, batch_size, n_epochs, score_fn, loss_fn, optimizer, 
        lr_scheduler=None, evaluate=True)
    results.append(partial_results)

In [ ]:
# === selecting best LR scheduler ===

models = [
    fast_scnn.FastSCNN(n_classes),
    fast_scnn.FastSCNN(n_classes),
    fast_scnn.FastSCNN(n_classes),
]

results = []
loss_fn = nn.CrossEntropyLoss()

optimizers = [
    torch.optim.Adam(models[0].parameters(), lr=learning_rate),
    torch.optim.Adam(models[1].parameters(), lr=learning_rate),
    torch.optim.Adam(models[2].parameters(), lr=learning_rate)]

scheduler_candidates = [
    None,
    torch.optim.lr_scheduler.LinearLR(optimizers[1]),
    torch.optim.lr_scheduler.ExponentialLR(optimizers[2], gamma=0.1)]

for i, scheduler in enumerate(scheduler_candidates):
    model = models[i]
    optimizer = optimizers[i]

    partial_results = training_and_testing.train_model(
        device, model, train_dataset, batch_size, n_epochs, score_fn, loss_fn, optimizer, 
        lr_scheduler=scheduler, evaluate=True)
    results.append(partial_results)